In [1]:
# Set the working directory to the parent directory
import sys
sys.path.append('..')
sys.dont_write_bytecode = True

import os
os.chdir("..")

# Import relevant packages
from src.eda import data_info
from src.explainability import ShapKernelExplainer

# Import necessary libraries
import shap
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import pandas as pd

In [ ]:
version = "202505112013"
model_type = "baseline"

shap_init = ShapKernelExplainer(model_type=model_type)
shap_init.explain_all_versions()

Explaining anomalies for baseline model version: 202505080154:   3%|▎         | 19/574 [00:17<08:32,  1.08it/s]